In [1]:
#Bibliotecas necessárias p/ Projeto
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, regexp_replace, UserDefinedFunction
from pyspark.sql.types import ByteType,DataType,FloatType,IntegerType,StructType,StructField
import sqlalchemy


In [2]:
spark = SparkSession.builder.appName('Yandeh_Project').getOrCreate()

In [3]:
#Lendo arquivos CSV e salvando o Dataframe usando Pandas
pd_cotacao_preco = pd.read_csv('cotacao_precos.csv',encoding='latin1',sep=';')
pd_espec_componentes = pd.read_csv('espec_componentes.csv',encoding='latin1',sep=';')
pd_lista_materiais = pd.read_csv('lista_materiais.csv',encoding='latin1',sep=';')

In [4]:
#Usando Spark
#cotacao_preco_schema = Struc
df_cotacao_preco = spark.read.option("sep",";").option("header",True).option("encoding","latin1").csv('cotacao_precos.csv')
df_espec_componentes = spark.read.option("sep",";").option("header",True).option("encoding","latin1").csv('espec_componentes.csv')
df_lista_materiais = spark.read.option("sep",";").option("header",True).option("encoding","latin1").csv('lista_materiais.csv')

<h1> Tabela <b>Cotação_Preço </b> </h1> 

In [15]:
#Tratando tipo das Colunas
df_cotacao_preco_tratado = df_cotacao_preco.withColumn('bico_injetor_id',F.col('bico_injetor_id').cast('string')).withColumn('data_cotacao',col('data_cotacao').cast('date')).withColumn('uso_anual',col('uso_anual').cast('int')).withColumn('qtde_min_pedido',col('qtde_min_pedido').cast('int')).withColumn('desconto_qtde',F.when(df_cotacao_preco['desconto_qtde']=="Sim",1).otherwise(0)).withColumnRenamed('quanitdade','quantidade').withColumn('quantidade',F.col('quantidade').cast('int')).withColumn('custo',F.col('custo').cast('float'))
df_cotacao_preco_tratado = df_cotacao_preco_tratado.withColumn('desconto_qtde',F.col('desconto_qtde').cast('int'))

<h1> Tabela <b>Lista Materias</b> </h1>

In [7]:
#Tratando tipo das Colunas
df_lista_materiais_tratado = df_lista_materiais.withColumn('bico_injetor_id',F.col('bico_injetor_id').cast('string')).withColumn('componente_id_1',F.col('componente_id_1').cast('string')).withColumn('componente_id_2',F.col('componente_id_2').cast('string')).withColumn('componente_id_3',F.col('componente_id_3').cast('string')).withColumn('componente_id_4',F.col('componente_id_4').cast('string')).withColumn('componente_id_5',F.col('componente_id_5').cast('string')).withColumn('componente_id_6',F.col('componente_id_6').cast('string')).withColumn('componente_id_7',F.col('componente_id_7').cast('string')).withColumn('componente_id_8',F.col('componente_id_8').cast('string')).withColumn('quantidade_1',F.col('quantidade_1').cast('int')).withColumn('quantidade_2',F.col('quantidade_2').cast('int')).withColumn('quantidade_3',F.col('quantidade_3').cast('int')).withColumn('quantidade_4',F.col('quantidade_4').cast('int')).withColumn('quantidade_5',F.col('quantidade_5').cast('int')).withColumn('quantidade_6',F.col('quantidade_6').cast('int')).withColumn('quantidade_7',F.col('quantidade_7').cast('int')).withColumn('quantidade_8',F.col('quantidade_8').cast('int'))

<h1> Tabela <b>Espec_Componentes</b> </h1>

In [ ]:
#transformar NA em Null
df_espec_componentes_tratado = df_espec_componentes.withColumn('componente_id',F.col('componente_id').cast('string')).withColumn('componente_tipo_id',F.col('componente_tipo_id').cast('string')).withColumn('tipo',F.col('tipo').cast('string')).withColumn('conexao_tipo_id',F.col('conexao_tipo_id').cast('string')).withColumn('forma',F.col('forma').cast('string')).withColumn('tipo_base',F.col('tipo_base').cast('string')).withColumn('distancia_base',F.col('distancia_base').cast('float')).withColumn('compr_padrao_parafuso',F.col('compr_padrao_parafuso').cast('float')).withColumn('larg_padrao_parafuso',F.col('larg_padrao_parafuso').cast('float')).withColumn('sulco',F.when(df_espec_componentes['sulco']=="Sim",1).otherwise(0)).withColumn('diametro_base',F.col('diametro_base').cast('float')).withColumn('diametro_borda',F.col('diametro_borda').cast('float')).withColumn('especificidade',F.when(df_espec_componentes['especificidade']=="Sim",1).otherwise(0)).withColumn('orientatacao',F.when(df_espec_componentes['orientatacao']=="Sim",1).otherwise(0)).withColumn('peso',F.col('peso').cast('float'))
df_espec_componentes_tratado = df_espec_componentes_tratado.withColumn('sulco',col('sulco').cast('boolean')).withColumn('especificidade',col('especificidade').cast('boolean')).withColumn('orientatacao',col('orientatacao').cast('boolean'))

<h1>Ingestão de Dados</h1>

In [30]:
df_espec_componentes_tratado.write.format('jdbc').options(
      url='jdbc:mysql://mysqlserver:3306/Yandeh?useTimezone=true&serverTimezone=UTC',
      driver='com.mysql.jdbc.Driver',
      dbtable='espec_Componente',
      user='root',
      password='senha!').mode('append').save()

In [31]:
df_cotacao_preco_tratado.write.format('jdbc').options(
      url='jdbc:mysql://mysqlserver:3306/Yandeh?useTimezone=true&serverTimezone=UTC',
      driver='com.mysql.jdbc.Driver',
      dbtable='cotacao_Preco',
      user='root',
      password='senha!').mode('append').save()

In [32]:
df_lista_materiais_tratado.write.format('jdbc').options(
      url='jdbc:mysql://mysqlserver:3306/Yandeh?useTimezone=true&serverTimezone=UTC',
      driver='com.mysql.jdbc.Driver',
      dbtable='lista_Material',
      user='root',
      password='senha!').mode('append').save()